# 서울특별시 공공자전거 따릉이 대여이력 분석
* [miningful/seoulbike: 따릉이 따세권EDA](https://github.com/miningful/seoulbike)에서 흥미를 얻어 분석을 시작하게 됨

## 데이터 출처 :
* 서울특별시 공공자전거 대여이력 정보
http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-15182&srvType=F&serviceKind=1&currentPageNo=1

* `(이달의주요공개정보)_서울_공공자전거(따릉이)_이용현황 자료(161021).xlsx (1.24MB)`
https://opengov.seoul.go.kr/anspruch/10045476


## 관련자료 
* [(석간) 서울시 따릉이 회원 62만 돌파 출 퇴근시간 38% 집중](http://spp.seoul.go.kr/main/news/news_report.jsp#view/253821)
* [공공자전거 따릉이 인기 따라 '따세권' 형성…소외지역도 :: 공감언론 뉴시스통신사 ::](http://www.newsis.com/view/?id=NISX20180706_0000356247&cID=10201&pID=10200)
* [서울특별시 빅데이터 캠퍼스 > 분석결과/사례공유 > 서울시 분석사례 > 서울시 자전거 이동경로 분석](https://bigdata.seoul.go.kr/noti/selectNoti.do?r_id=P430&bbs_seq=229&sch_type=&sch_text=&currentPage=1)
* [따릉이 편리하게 이용하기 위한 14가지 | 서울시 정보소통광장(정보공개)](https://opengov.seoul.go.kr/mediahub/15085803)

In [ ]:
# 노트북이 간결해 보이도록 경고 문구는 출력되지 않도록 함
# 프로그래밍에 익숙하다면 경고 문구도 출력해서 보도록 한다. 
# 하지만 익숙하지 않다면 당황하지 않기 위해 경고 메시지는 찍히지 않도록 한다.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 사용할 도구 가져오기

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns


# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

## 데이터 불러오기
* 서울시정보소통광장에서 다운로드 받은 csv파일과 크롤링해온 대여소 정보가 있다.
* 이 노트북에서는 전체 데이터 중 일부 데이터만을 사용해서 특정 시기의 데이터만 분석해 보고 점점 더 큰 데이터를 불러와 보도록 한다.

In [ ]:
%ls data/

In [ ]:
df = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv', encoding='cp949')
# 2018년 2분기까지의 데이터를 보고 싶다면 아래 파일을 불러온다.
# %time df = pd.read_csv('data/seoul-bike-2018.csv')
df.shape

## 데이터 미리보기 및 요약하기
* head와 tail을 통해 데이터 미리보기를 해볼 수 있다.
* head는 데이터의 윗부분을 tail은 데이터의 하단부를 미리보기 해준다. 
* 괄호안에 숫자를 넣어주면 해당 숫자만큼의 데이터를 가져온다. 기본값은 5이다.

In [ ]:
df.head()

In [ ]:
df.tail()

* info를 사용하면 해당 데이터의 요약정보를 볼 수 있다.
* 데이터의 갯수와 데이터의 타입, 크기 등을 볼 수 있다.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include=np.object)

## 분석을 위한 전처리
* 위에서 데이터를 미리보기 했을 때 컬럼과 데이터에 불필요한 홑따옴표가 포함되어 있다.
* 전처리를 통해 제거해 준다.
* object 타입의 데이터에는 모두 홑따옴표가 들어가 있다. 따라서 object 데이터 타입이라면 홑따옴표를 제거해 주도록 한다.

In [ ]:
# 컬럼에 있는 '를 제거한다.
df.columns = df.columns.str.strip("'")

In [ ]:
# 불필요하게 '가 들어가 있다. 스트링 타입으로 되어있는 데이터에서 '을 제거하자.
# 2018년 전체 데이터를 모아놓은 csv에서는 미리 제거해서 넣어두었다.
df = df.apply(lambda x: x.str.strip("'") if x.dtype == np.dtype('object') else x)

# 다음의 방법으로 제거할 수도 있다.
# df.iloc[:, :4] = df.iloc[:, :4].apply(lambda x: x.str.strip("'"))
# df.iloc[:, 5:8] = df.iloc[:, 5:8].apply(lambda x: x.str.strip("'"))

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe(include=np.object)

## 결측치 보기

In [ ]:
df.isnull().sum()

## 결측치 시각화 하기
* missingno는 별도로 설치해야 하는 시각화 도구 입니다. 결측치를 시각화해서 볼 수 있습니다.

In [ ]:
import matplotlib.pyplot as plt

# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 

In [ ]:
# 결측치 보기
import missingno as msno

msno.matrix(df, figsize=(12,5))

## 대여소별 대여|반납 현황

In [ ]:
print('# 대여소 수')
print('대여 대여소: ', df['대여대여소번호'].unique().shape[0])
print('반납 대여소: ', df['반납대여소번호'].unique().shape[0])

In [ ]:
df['대여대여소번호'].value_counts().tail(10)

* 대여 대여소번호에 상암센터 정비실과 위트콤 같은 숫자가 아닌 곳들이 있음

In [ ]:
# 대여량이 많은 상위 대여소
df['대여대여소명'].value_counts().head(10)

In [ ]:
# 반납량이 많은 상위 대여소
df['반납대여소명'].value_counts().head(10)

1. 대여반납이 많은 상위 대여소인 홍대입구역, 여의나루역, 뚝섬유원지, 롯데월드타워는 모두 강이나 호수를 인근에 두고 있다. 
2. 성대입구나 안암로터리 버스정류장은 대학가 인근이다.

In [ ]:
# 평균 이용거리가 긴 상위 대여소
df.groupby(['대여대여소명'])['이용거리(M)'].mean().reset_index().sort_values(
    by='이용거리(M)', ascending=False).head(10)

In [ ]:
# 평균 이용거리가 짧은 대여소
df.groupby(['대여대여소명'])['이용거리(M)'].mean().reset_index().sort_values(
    by='이용거리(M)', ascending=True).head()

In [ ]:
# 평균 이용시간이 많은 상위 대여소
df.groupby(['대여대여소명'])['이용시간(분)'].mean().reset_index().sort_values(
    by='이용시간(분)', ascending=False).head()

In [ ]:
# 평균 이용시간이 적은 상위 대여소
df.groupby(['대여대여소명'])['이용시간(분)'].mean().reset_index().sort_values(
    by='이용시간(분)', ascending=True).head()

## 시계열 데이터 보기

In [ ]:
df['대여일시'] = pd.to_datetime(df['대여일시'])
df['반납일시'] = pd.to_datetime(df['반납일시'])
df.dtypes

In [ ]:
df['대여년월일'] = df['대여일시'].dt.date
df['반납년월일'] = df['반납일시'].dt.date
df['대여년'] = df['대여일시'].dt.year
df['대여월'] = df['대여일시'].dt.month
df['대여일'] = df['대여일시'].dt.day
df['대여시'] = df['대여일시'].dt.hour
df['대여분'] = df['대여일시'].dt.minute
df['대여요일'] = df['대여일시'].dt.dayofweek
df.columns

In [ ]:
df['반납년월일'] = df['반납일시'].dt.date
df['반납년월일'] = df['반납일시'].dt.date
df['반납년'] = df['반납일시'].dt.year
df['반납월'] = df['반납일시'].dt.month
df['반납일'] = df['반납일시'].dt.day
df['반납시'] = df['반납일시'].dt.hour
df['반납분'] = df['반납일시'].dt.minute
df['반납요일'] = df['반납일시'].dt.dayofweek
df.columns

In [ ]:
df.head()

In [ ]:
df_month_1 = df['대여월'].value_counts().reset_index()
df_month_1.columns = ['month', 'count']

df_month_1 = df_month_1.sort_values('month')
df_month_1['대여반납'] = '대여'
print(df_month_1.shape)

df_month_2 = df['반납월'].value_counts().reset_index()
df_month_2.columns = ['month', 'count']
df_month_2 = df_month_2.sort_values('month')
df_month_2['대여반납'] = '반납'
print(df_month_2.shape)

df_month = pd.concat([df_month_1, df_month_2])
print(df_month.shape)
df_month.head()

In [ ]:
df_month_pivot =  df_month.pivot_table(index=['month'],
                                       columns=['대여반납'], 
                                       values='count', aggfunc=sum)
df_month_pivot

In [ ]:
df_month_pivot.plot.bar(title='월별 자전거 대여수', rot=0)

In [ ]:
sns.barplot(data=df_month, x='month', y='count', hue='대여반납')

In [ ]:
# (ggplot(df_month)
#  + aes(x='month', y='count', fill='대여반납')
#  + geom_bar(stat='identity', position='dodge')
#  + labs(x='대여월', y='대여수', title='월별 자전거 대여수')
#  + theme(text=element_text(family='NanumBarunGothic'),
#         figure_size=(12,6))
# )

## 일자별 대여|반납 현황
* 날짜에 따라 대여|반납 횟수를 확인한다.

In [ ]:
df_day_1 = df['대여일'].value_counts().reset_index()
df_day_1.columns = ['day', 'count']

df_day_1 = df_day_1.sort_values('day')
df_day_1['대여반납'] = '대여'
print(df_day_1.shape)

df_day_2 = df['반납일'].value_counts().reset_index()
df_day_2.columns = ['day', 'count']
df_day_2 = df_day_2.sort_values('day')
df_day_2['대여반납'] = '반납'
print(df_day_2.shape)

df_day = pd.concat([df_day_1, df_day_2])
print(df_day.shape)
df_day.head()

In [ ]:
df_day_pivot =  df_day.pivot_table(index=['day'],
                                       columns=['대여반납'], 
                                       values='count', aggfunc=sum)
df_day_pivot.head()

In [ ]:
df_day_pivot.plot.bar(title='일별 자전거 대여수', rot=0, figsize=(20, 6))

In [ ]:
sns.barplot(data=df_day, x='day', y='count', hue='대여반납')

In [ ]:
# (ggplot(df_day)
#  + aes(x='day', y='count', fill='대여반납')
#  + geom_bar(stat='identity', position='dodge')
#  + labs(x='대여일', y='대여수', title='일별 자전거 대여수')
#  + theme(text=element_text(family='NanumBarunGothic'),
#         figure_size=(12,6))
# )

In [ ]:
df_ymd_1 = df['대여년월일'].value_counts().reset_index()
df_ymd_1.columns = ['ymd', 'count']

df_ymd_1 = df_ymd_1.sort_values('ymd')
df_ymd_1['대여반납'] = '대여'
print(df_ymd_1.shape)

df_ymd_2 = df['반납년월일'].value_counts().reset_index()
df_ymd_2.columns = ['ymd', 'count']
df_ymd_2 = df_ymd_2.sort_values('ymd')
df_ymd_2['대여반납'] = '반납'
print(df_ymd_2.shape)

df_ymd = pd.concat([df_ymd_1, df_ymd_2])
print(df_ymd.shape)
df_ymd.head()

In [ ]:
# 시간데이터로 표현하기 위해 타입을 바꿔준다.
df_ymd["ymd"] = pd.to_datetime(df_ymd["ymd"])
df_ymd["yymmdd"] = df_ymd["ymd"].dt.date

In [ ]:
plt.figure(figsize=(20, 5))
plt.xticks(rotation=30, ha='right')
sns.pointplot(data=df_ymd, x='yymmdd', y='count')

In [ ]:
(ggplot(df_ymd)
 + aes(x='ymd', y='count', fill="대여반납")
 + geom_point()
 + geom_line(color='blue')
 + theme(text=element_text(family='NanumBarunGothic'),
        figure_size=(12,6))
)

In [ ]:
# 최근 30일치 데이터를 가져온다. 대여반납 2가지 데이터가 있기 때문에 -60으로 30일치의 데이터를 가져온다.
(ggplot(df_ymd[-60:])
 + aes(x='ymd', y='count', fill="대여반납")
 + geom_point()
 + geom_line(color='blue')
 + theme(text=element_text(family='NanumBarunGothic'),
        figure_size=(12,6))
)

* 일부 날짜의 데이터만 있다.
* 2018년 6월 데이터만 봤을 때 13, 26일에는 대여와 반납모두 적다.
* 26일의 일기예보를 보면 장맛비가 내린것을 확인해 볼 수 있다. https://www.youtube.com/watch?v=cYHC8iwfFyU

## 요일별 대여|반납 현황
* 대여|반납별로 어떤 요일에 주로 대여반납이 되는지 시각화해보기 위해 Tidy Data를 만든다.

In [ ]:
df_dow_1 = df['대여요일'].value_counts().reset_index()
df_dow_1.columns = ['dayofweek', 'count']
df_dow_1 = df_dow_1.sort_values('dayofweek')
df_dow_1['대여반납'] = '대여'
print(df_dow_1.shape)

df_dow_2 = df['반납요일'].value_counts().reset_index()
df_dow_2.columns = ['dayofweek', 'count']
df_dow_2 = df_dow_2.sort_values('dayofweek')
df_dow_2['대여반납'] = '반납'
print(df_dow_2.shape)

df_dow = pd.concat([df_dow_1, df_dow_2])
print(df_dow.shape)

In [ ]:
df_dow.head(3)

In [ ]:
df_dow.tail(3)

In [ ]:
weekday_map= {0:'MON', 1:'TUE', 2:'WED', 3:'THU',
              4:'FRI', 5:'SAT', 6:'SUN'}

In [ ]:
(ggplot(df_dow)
 + aes(x='dayofweek', y='count', fill='대여반납')
 + geom_bar(stat='identity', position='dodge')
 + geom_point()
 + geom_line(color='blue')
 + labs(x='대여요일', y='대여수', title='요일별 자전거 대여수')
 + theme(text=element_text(family='NanumBarunGothic'),
        figure_size=(12,6))
)

## 시간대별 대여|반납량

In [ ]:
df_hour_1 = df['대여시'].value_counts().reset_index()
df_hour_1.columns = ['hour', 'count']
df_hour_1 = df_hour_1.sort_values('hour')
df_hour_1['대여반납'] = '대여'
df_hour_1.head()

df_hour_2 = df['반납시'].value_counts().reset_index()
df_hour_2.columns = ['hour', 'count']
df_hour_2 = df_hour_2.sort_values('hour')
df_hour_2['대여반납'] = '반납'
df_hour_2.head()

df_hour = pd.concat([df_hour_1, df_hour_2])
df_hour.head()

In [ ]:
(ggplot(df_hour)
 + aes(x='hour', y='count', fill='대여반납')
 + geom_bar(stat='identity', position='dodge')
 + geom_point()
 + geom_line(color='blue')
 + labs(x='시간', y='대여/반납 수', title='시간대별 자전거 대여/반납수')
 + theme(text=element_text(family='NanumBarunGothic'),
        figure_size=(12,6))
)

* 새벽시간대로 갈 수록 대여/반납 수가 줄어든다.
* 출근시간대에는 대여가 반납보다 많다.
* 오전에는 8시대에 대여/반납이 많으며, 오후시간대에는 18시에 대여/반납이 가장 많다.
* 오전 11시부터 20시까지는 대여가 반납보다 많다.(대여>반납)
* 21시 이후부터 오전 4시까지는 대여보다 반납이 많다.(대여<반납)

In [ ]:
df_minute_1 = df['대여분'].value_counts().reset_index()
df_minute_1.columns = ['minute', 'count']
df_minute_1 = df_minute_1.sort_values('minute')
df_minute_1['대여반납'] = '대여'
print(df_minute_1.shape)

df_minute_2 = df['반납분'].value_counts().reset_index()
df_minute_2.columns = ['minute', 'count']
df_minute_2 = df_minute_2.sort_values('minute')
df_minute_2['대여반납'] = '반납'
print(df_minute_2.shape)

df_minute = pd.concat([df_minute_1, df_minute_2])
print(df_minute.shape)

In [ ]:
(ggplot(df_minute)
 + aes(x='minute', y='count', fill='대여반납')
 + geom_bar(stat='identity', position='dodge')
 + geom_line(color='blue')
 + labs(x='분', y='대여/반납 수', title='시간대별(분) 자전거 대여/반납수')
 + theme(text=element_text(family='NanumBarunGothic'),
        figure_size=(12,6))
)

* 대여/반납이 거의 비슷하게 이루어지고 있지만 대여는 0~20분 사이에 더 많이 일어나며 반납은 40~60분 사이에 더 많이 일어난다.

## 대여와 반납은 같은 곳에서 이루어지고 있을까?

In [ ]:
# 같은대여반납소라는 컬럼을 새로 만들고 대여반납이 같은 곳에서 이루어지고 있는지 여부를 추가한다.
df['같은대여반납소'] = df['대여대여소명'] == df['반납대여소명']

In [ ]:
df_location_diff = df['같은대여반납소'].value_counts().reset_index()
df_location_diff.columns = ['일치여부', '대여반납수']
# 같은|다른 반납소별로 대여반납수가 몇 배나 차이가 나는지 확인한다.
location_diff_ratio = df_location_diff['대여반납수'][0]/df_location_diff['대여반납수'][1]
print('같은 곳에서 대여반납 하는 것보다 다른 곳에서 대여반납 하는 것이 {0:.2f}배 많다.'.format(location_diff_ratio))
df_location_diff

In [ ]:
# 대여반납수로 fill을 넣어주어 그래프에 색상표현이 되도록 한다.
(ggplot(df_location_diff)
 + geom_col(mapping=aes(x='일치여부', y='대여반납수', fill='대여반납수'))
 + labs(title='대여반납소 일치여부')
 + theme(text=element_text(family='NanumBarunGothic'))
)

같은 곳에서 대여반납을 하는 것보다 다른 곳으로의 대여반납이 훨씬 많은 것을 확인할 수 있다.

In [ ]:
df_same_loc = df.loc[df['같은대여반납소'] == True]
df_same_loc.head()

In [ ]:
# 같은 곳에서 대여반납이 이루어지는 상위 대여소(대여|반납 건 수)
df_same_loc['대여대여소명'].value_counts().head(30)

* 같은 곳에서 대여반납이 이루어지는 대여소들은 한강주변이나 공원 주변에 위치한 것을 확인해 볼 수 있다.

## 같은|다른 대여소에서 대여가 되는 자전거의 평균 이용시간
### 같은 대여반납소

In [ ]:
same_loc_mean_time = df_same_loc['이용시간(분)'].mean()
print('같은곳에서 대여, 반납이 이루어지는 자전거의 평균 이용시간 : ', same_loc_mean_time)
df_same_mean = df_same_loc.groupby(['대여대여소명', '반납대여소명'])['이용시간(분)'].mean().reset_index()
df_same_count = df_same_loc.groupby(['대여대여소명', '반납대여소명']).size().reset_index()
df_same_count.columns = ['대여대여소명', '반납대여소명', '이용횟수']
df_same_loc2 = df_same_mean.merge(df_same_count, left_on=['대여대여소명', '반납대여소명'], 
                                  right_on=['대여대여소명', '반납대여소명'])
df_same_loc2.sort_values(by='이용시간(분)', ascending=False).head(10)

In [ ]:
# 이용횟수가 많은 곳의 평균 이용시간
df_same_loc2.sort_values(by='이용횟수', ascending=False).head(10)

In [ ]:
df_same_loc2['이용횟수'].plot.hist()

### 다른 대여|반납소

In [ ]:
df_diff_loc = df.loc[df['같은대여반납소'] == False]
df_diff_loc.shape

In [ ]:
diff_loc_mean_time = df_diff_loc['이용시간(분)'].mean()
print('다른곳에서 대여, 반납이 이루어지는 자전거의 평균 이용시간 : ', df_diff_loc['이용시간(분)'].mean())
df_diff_mean = df_diff_loc.groupby(['대여대여소명', '반납대여소명'])['이용시간(분)'].mean().reset_index()
df_diff_count = df_diff_loc.groupby(['대여대여소명', '반납대여소명']).size().reset_index()
df_diff_count.columns = ['대여대여소명', '반납대여소명', '이용횟수']
df_diff_loc2 = df_diff_mean.merge(df_diff_count, 
                                  left_on=['대여대여소명', '반납대여소명'], 
                                  right_on=['대여대여소명', '반납대여소명'])
df_diff_loc2.sort_values(by='이용시간(분)', ascending=False).head(10)

In [ ]:
df_diff_loc2.sort_values(by='이용횟수', ascending=False).head(10)

In [ ]:
rent_mean_time = same_loc_mean_time - diff_loc_mean_time
print('같은 곳에서 대여반납이 이루어지는 자전거의 이용시간(분)이 다른 대여소의 대여반납 이용시간보다 {:,.2f} 분 많다.'.format(rent_mean_time))

* 이용횟수가 많은 대여소라고 해서 평균이용시간이 평균보다 높지는 않다.
* 하늘채코오롱아파트 건너편<->홍대입구역 2번출구 앞은 이용횟수가 가장 높은 대여반납소다.
* 관련기사
    * ['따릉이' 타고 가장 많이 다닌 경로는? - 통플러스](http://news.tongplus.com/site/data/html_dir/2018/10/24/2018102402585.html)
    * [[ONE SHOT] 따릉이, ‘이곳’서 가장 많이 빌리고 ‘여기’서 가장 많이 탄다](https://news.joins.com/article/23003297)

## 해당 기간동안 가장 많이 이용된 자전거는 몇 회 대여 되었을까?

In [ ]:
bike_describe = df['자전거번호'].describe()
print('대여된 자전거 수 :', bike_describe[1])
print('가장 많이 대여된 자전거 번호 :', bike_describe[2])
print('가장 많이 대여된 자전거의 대여횟수 :', bike_describe[3])
bike_describe

In [ ]:
bike_rent_counts = df['자전거번호'].value_counts().reset_index()
bike_rent_counts.columns = ['자전거번호', '대여수']
print('해당 기간동안 자전거 하나당 평균 대여 수:', bike_rent_counts['대여수'].mean())
print('자전거 하나당 가장 많이 대여된 횟수:', bike_rent_counts['대여수'].max())
print('자전거 하나당 가장 적게 대여된 횟수:', bike_rent_counts['대여수'].min())

In [ ]:
bike_rent_counts.plot.hist()

In [ ]:
# 가장 많이 대여된 자전거의 대여이력
most_bike_df = df.loc[df['자전거번호'] == bike_describe[2]]
most_bike_df.head()

In [ ]:
most_bike_df['대여대여소명'].value_counts()

In [ ]:
rent_station = most_bike_df['대여대여소명'].unique()
rent_station

In [ ]:
# 대여소명 앞에 들어간 공백을 제거합니다.
bike_move = []
for s in rent_station:
    bike_move.append(s.strip())
    
bike_move    

In [ ]:
stations = pd.read_csv('data/bike_rent_station.csv')
stations.shape

In [ ]:
bike_move_df = pd.DataFrame(bike_move)
bike_move_df.columns = ['대여소']
print(bike_move_df.shape)
bike_move_df.head()

In [ ]:
bike_move_df = bike_move_df.merge(stations, left_on='대여소', right_on='대여소', how='left')
bike_move_df.shape

In [ ]:
bike_move_df.head()

In [ ]:
bike_move_df.isnull().sum()

## 가장 많이 대여된 자전거가 거쳐간 대여소

In [ ]:
# 정보가 없는 대여소
bike_move_df.loc[bike_move_df['경도'].isnull()]

In [ ]:
# 지도 표현을 위해 folium 을 import 한다.
# 데이터 프레임에서 위도와 경도의 평균값을 구한다. 이 값으로 지도의 중심이 될 좌표를 정한다.
# zoom_start는 지도를 얼마나 확대해서 볼것인지다.
# tiles는 지도의 스타일이다.
import folium 

geo_df = bike_move_df
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], zoom_start=12, tiles='Stamen Toner')

for n in geo_df.index:
    if geo_df.loc[n, '위도'] > 0 and geo_df.loc[n, '경도'] > 0 :
        loc_name = geo_df.loc[n, '대여소']

        icon_color = 'red'
        folium.Circle(
            location=[geo_df.loc[n, '위도'], geo_df.loc[n, '경도']],
            popup=loc_name,
            radius=50,
            color=icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)
    
map